In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
%matplotlib nbagg

In [38]:
with open("datasets/train.json", encoding = "UTF8") as data_file:
    train_data = json.load(data_file)
with open("datasets/test.json", encoding = "UTF8") as data_file:
    test_data = json.load(data_file)

In [40]:
train_data = pd.DataFrame(train_data)
test_data = pd.DataFrame(test_data)

In [4]:
train_data.shape

(39774, 3)

In [5]:
test_data.shape

(9944, 2)

In [6]:
x_set = set()
for ingredient in train_data["ingredients"]:
    x_set = x_set.union(set(ingredient))
x_set = list(x_set)

In [7]:
len(x_set)

6714

In [42]:
train_new_data = train_data
for ingredient in x_set:
    train_new_data[ingredient] = 0

In [9]:
train_data.loc[1,"ingredients"]

['plain flour',
 'ground pepper',
 'salt',
 'tomatoes',
 'ground black pepper',
 'thyme',
 'eggs',
 'green tomatoes',
 'yellow corn meal',
 'milk',
 'vegetable oil']

In [10]:
for row in range(len(train_data)):
    for ingredient in train_new_data.loc[row,"ingredients"]:
        train_new_data.loc[row,ingredient] = 1
train_new_data = train_new_data.drop(columns = ["ingredients"])

In [11]:
# from sklearn.model_selection import train_test_split as tts

y_col = "cuisine"
X_col = train_new_data.columns.drop([y_col, "id"])
# X_train, X_test, y_train, y_test = tts(train_new_data[X_col], train_new_data[y_col], test_size = 0.3, random_state = 42)
X_train, y_train = train_new_data[X_col], train_new_data[y_col]

In [13]:
from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import confusion_matrix

model_lr = LogisticRegression()
model_lr.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [14]:
from sklearn.model_selection import cross_val_score

cross_val_score(model_lr, X_train, y_train, cv=5, scoring="accuracy")

array([0.77354936, 0.78461925, 0.77991453, 0.77675764, 0.78482446])

In [43]:
test_new_data = test_data
for ingredient in x_set:
    test_new_data[ingredient] = 0

In [49]:
test_new_data.head()

,id,flounder fillets,prebaked pizza crusts,frozen mashed potatoes,Texas Pete Hot Sauce,crab sticks,soy bean paste,pimenton,Pernod Liqueur,steamed white rice,...,shrimp tails,tangerine,halibut fillets,salted seaweed,crosswise,chocolate leaves,coconut rum,Splenda Brown Sugar Blend,knorr chicken flavor bouillon cube,pepperoni slices
0,18009,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,28583,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,41580,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,29752,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,35687,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [46]:
for row in range(len(test_data)):
    for ingredient in test_new_data.loc[row,"ingredients"]:
        if ingredient in test_new_data.columns:
            test_new_data.loc[row,ingredient] = 1
test_new_data = test_new_data.drop(columns = ["ingredients"])

In [50]:
X_test = test_new_data.drop(columns = ["id"])

In [51]:
y_test = model_lr.predict(X_test)

In [91]:
subs = pd.read_csv("datasets/sample_submission.csv")

In [92]:
subm = subs.copy()
subm["cuisine"] = y_test

In [94]:
pd.DataFrame.to_csv(subm, "datasets/submission_logistic.csv", index=False)

,id,cuisine
0,35203,italian
1,17600,italian
2,35200,italian
3,17602,italian
4,17605,italian


In [95]:
from sklearn.ensemble import RandomForestClassifier

model_rf = RandomForestClassifier(n_jobs=-1, random_state=42)
model_rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [19]:
cross_val_score(model_rf, X_train, y_train, cv=5, scoring="accuracy")

array([0.66679226, 0.6734104 , 0.66226747, 0.66218086, 0.67346168])

In [96]:
y_test = model_rf.predict(X_test)
subrf = subs.copy()
subrf["cuisine"] = y_test

In [97]:
pd.DataFrame.to_csv(subrf, "datasets/submission_rf.csv", index=False)